In [1]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [2]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

D:\pycharm_project\vs2ttsvoice\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder\saved_models\pretrained\pretrained.pt


In [14]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed,_ = encoder.embed_utterance(preprocessed_wav)
    return embed

In [4]:
class X2E_NetSmooth(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,out_dim):
        super(X2E_NetSmooth, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))  
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, n_hidden_3), torch.nn.ReLU(True))
        self.layer4 = torch.nn.Sequential(torch.nn.Linear(n_hidden_3, n_hidden_4), torch.nn.ReLU(True))
        self.layer5 = torch.nn.Sequential(torch.nn.Linear(n_hidden_4, out_dim))
 
    def forward(self, x):
        x=x/torch.norm(x, p=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x=x/torch.norm(x, p=2)
        return x

In [8]:
#模型
model=X2E_NetSmooth(512,450,400,350,300,256)
#损失函数
# criterion = torch.nn.MSELoss(reduce=True, size_average=False)
criterion = torch.nn.SmoothL1Loss()
# criterion = torch.nn.L1Loss(reduction='sum')
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
# 训练模型
epoch = 0
totaldir='D:\\pycharm_project\\vs2ttsvoice\\X2E'
#损失值写入文件

for i in range(80):
    pathstr ='xvector'+str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir, pathstr)
    file=open( pathstr)
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        filename=data[0]
        data=data[1:]
        out=list(map(float,data))
        out=torch.tensor(out)
        pred=model(out)
        label=embedEX(filename)
        label=torch.from_numpy(label)
        loss = criterion(pred, label)
        print_loss = loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch+=1
        if epoch%100 == 0:
        #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
        #保存参数、
            print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
            state = { 'model':model.state_dict(), 'epoch':epoch }                       
            if not os.path.isdir('checkpointX2E'):
                 os.mkdir('checkpointX2E')    
            torch.save(state, '.\checkpointX2E\modelSmooth_epoch_%d.ckpt' % (epoch))
            file_save = open('lossSmooth.txt',mode='a')
            file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
            file_save.close() 

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\voiceset\\dataset\\test_wav\\id10001\\1zcIwhmdeo4\\00001.wav'

In [19]:
#验证合成的声音
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpointX2E\modelSmooth_epoch_148600.ckpt')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [ ]:
#合成语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open('.\X2E\pv_xvector.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
#     out=[[list(map(float,data))]]
#     out=torch.tensor(out)
#     out=out.permute(0,2,1)
#     pred=model(out)
#     pred=pred.squeeze(0)
#     pred=pred.tolist()
#     print("pred",pred)
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
#     print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
#     sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
#     sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

id10270-5r0dWxy17C8-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.607781264061873


id10271-1gtz-CUIygI-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6134702017876799


id10272-0qqQJVFKpbA-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6762868556847855


id10273-0LbtndiXJC0-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6371319150372436


id10274-7rkVfW2yw0c-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6263795009229111
